In [1]:
import requests
import re
import time
import pandas as pd
import json
import pymysql

In [3]:
def get_html(url):
    headers = {'Accpet':'*/*',
           'Accept-Encoding':'gzip, deflate, br',
          'Accept-Language':'zh-CN,zh;q=0.8,zh-TW;q=0.7,zh-HK;q=0.5,en-US;q=0.3,en;q=0.2',
          'Connection':'keep-alive',
          'Cookie':'t=9573c67d7fd6a557d0ac98513ef6c1f9; _tb_token_=fa9ff377afb3a; cookie2=102747c0ea62a9b91c0ca968cd1d26a3; cna=vWfuFYJnK3ACAbSgJxI4p8c3; isg=BAIC8YEn67qSZvdmBUy1xodHUAikeweKjq_nckwbH3UCn6MZNGFs_K2dT1tG1H6F; l=cB__pUSgqFq_65doKOCwNuI8L1799IObYuPRwCmXi_5Cl1LnUU7OkuDCvep6csWFToLB450m8zpTNFN0-y8foTB7K9cdvdC..; dnk=%5Cu767D%5Cu97ECbt; uc1=existShop=false&pas=0&cookie21=WqG3DMC9Fb5mPLIQo9kR&cookie14=UoTaH0e3TmMtJA%3D%3D&cookie16=V32FPkk%2FxXMk5UvIbNtImtMfJQ%3D%3D&lng=zh_CN&tag=8&cookie15=V32FPkk%2Fw0dUvg%3D%3D; uc3=lg2=UIHiLt3xD8xYTw%3D%3D&vt3=F8dBy3MJnq8w8iVXIWo%3D&nk2=05f%2B1%2F8c&id2=UUpie4zTYDrs2g%3D%3D; tracknick=%5Cu767D%5Cu97ECbt; lid=%E7%99%BD%E9%9F%ACbt; _l_g_=Ug%3D%3D; uc4=nk4=0%400Sx7w5PS%2B8XluSDCLoE%2Bmo4%3D&id4=0%40U2govm6nsY1s1lGE4V6n0Oo7vYeV; unb=2239688851; lgc=%5Cu767D%5Cu97ECbt; cookie1=WvZsE64Z%2BZ1nF5qtbK9eQz7Gn9kDhqYoohqmr5yXWPo%3D; login=true; cookie17=UUpie4zTYDrs2g%3D%3D; _nk_=%5Cu767D%5Cu97ECbt; sg=t1c; csg=ee88a3b5; x5sec=7b22726174656d616e616765723b32223a223836393437303832373562376262633230393037333661653562393133326237434f377a6e657346454b694a36752f633271715277414561444449794d7a6b324f4467344e5445374d513d3d227d; otherx=e%3D1%26p%3D*%26s%3D0%26c%3D0%26f%3D0%26g%3D0%26t%3D0; whl=-1%260%260%260; enc=F2o%2BPWkzGYV87p%2B5iBG2spulxPRek1CiXc1y72CiOEPB0AG0IlAGzrY942M71YMFBARDFbJ%2F2A8fuEHYbd7qSQ%3D%3D; x=__ll%3D-1%26_ato%3D0',
          'Host':'rate.tmall.com',
          'TE':'Trailers',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0',
          'Referer':'https://detail.tmall.com/'}
    response = requests.get(url,headers=headers)
    html = response.text
    return html

def get_data_onePage(html):
    json_temp = re.split('^\s\n\w+',html)[1].strip('()')
    data_temp = json.loads(json_temp)
    data_onePage = data_temp['rateDetail']['rateList']
    return data_onePage


def create_table(cursor):
    sql = "create table if not exists taobao_data(UserID varchar(255) not null, SellerID varchar(255) not null,\
    ReviewDate datetime not null, Review varchar(255) not null, Reply varchar(255) not null,\
    ReviewAdd varchar(255) not null, GoldUser varchar(255) not null, ItemID varchar(255) not null,primary key (userID))"

    cursor.execute(sql)
    print('-------->taobao_data创建成功<----------')
    
    
    
def save_in_MySQL(cursor,data,conn):
    sql = "insert into taobao_data(UserID, SellerID, ReviewDate, Review, Reply, ReviewAdd, GoldUser, ItemID)\
     values(%s,%s,%s,%s,%s,%s,%s,%s)"
    #在爬取时也命名为这些
    try:
        cursor.execute(sql,data)
        conn.commit()
        print('-------->数据提交成功<----------')
    except:
        conn.rollback()
        print('-------->数据提交失败，已撤销<----------')


def main(review_list,item_ID_list):
    #连接
    conn = pymysql.connect(host='localhost', user='root', password='123456', database='test1', port=3306, charset='utf8')
    cursor = conn.cursor()
    
    create_table(cursor)
    
    for i in range(len(item_ID_list)):
        page_num = review_list[i]//20  #一页默认20条评论
        item_ID = item_ID_list[i]
        
        for page in range(page_num):
            url_temp = 'https://rate.tmall.com/list_detail_rate.htm?itemId={}&spuId=1136187753&sellerId=196993935&order=3&currentPage={}&append=0&content=1'.format(item_ID,str(page+1))
            html = get_html(url_temp)
            data_onePage = get_data_onePage(html)
            #写入数据库
            
            for d in data_onePage:
                data = []
                data.append(d['id'])
                data.append(d['sellerId'])
                data.append(d['rateDate'])
                data.append(d['rateContent'])
                data.append(d['reply'])
                if d['appendComment'] is not None:  # 有的不含追加评论，需要判断。
                    data.append(d['appendComment']['content'])
                else:
                    data.append(' ')
                data.append(d['goldUser'])
                data.append(item_ID)
                
                save_in_MySQL(cursor,data,conn)
    
            print(page)
            time.sleep(2)
    #关闭连接
    cursor.close()
    conn.close()
    print('-------->本次爬取数据已全部入库<----------')


#修改item_ID_list、review_list运行
item_ID_list = [595133286060]
review_list = [3362]
main(review_list,item_ID_list)
print('-----------------over---------------')

C:\Users\Afterain\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1050, "Table 'taobao_data' already exists")
  result = self._query(query)


-------->taobao_data创建成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
0
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<-

-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
15
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<---------

-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
31
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<----------
-------->数据提交成功<---------